<a href="https://colab.research.google.com/github/wilson37wu/Tutorials/blob/master/quickstarts/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [20]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Audio Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Audio.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook provides an example of how to prompt Gemini Flash using an audio file. In this case, you'll use a [sound recording](https://www.jfklibrary.org/asset-viewer/archives/jfkwha-006) of President John F. Kennedy’s 1961 State of the Union address.

### Install dependencies

In [21]:
%pip install -q -U google-genai

In [22]:
from google import genai

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [23]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('Vibe_Coding_Test')

TimeoutException: Requesting secret Vibe_Coding_Test timed out. Secrets can only be fetched when running from the Colab UI.

## Upload an audio file with the File API

To use an audio file in your prompt, you must first upload it using the [File API](../quickstarts/File_API.ipynb).


In [ ]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"

In [ ]:
!wget -q $URL -O sample.mp3

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

your_file = client.files.upload(file='sample.mp3')

## Use the file in your prompt

In [ ]:
response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Listen carefully to the following audio file. Provide a brief summary.',
    your_file,
  ]
)

print(response.text)

## Inline Audio

For small requests you can inline the audio data into the request, like you can with images. Use PyDub to trim the first 10s of the audio:

In [ ]:
%pip install -Uq pydub

In [ ]:
from pydub import AudioSegment

In [ ]:
sound = AudioSegment.from_mp3("sample.mp3")

In [ ]:
sound[2000:12000] # slices are in ms

Add it to the list of parts in the prompt:

In [ ]:
from google.genai import types

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Describe this audio clip',
    types.Part.from_bytes(
      data=sound[:10000].export().read(),
      mime_type='audio/mp3',
    )
  ]
)

print(response.text)

Note the following about providing audio as inline data:

- The maximum request size is 20 MB, which includes text prompts, system instructions, and files provided inline. If your file's size will make the total request size exceed 20 MB, then [use the File API](https://ai.google.dev/gemini-api/docs/audio?lang=python#upload-audio) to upload files.
- If you're using an audio sample multiple times, it is more efficient to [use the File API](https://ai.google.dev/gemini-api/docs/audio?lang=python#upload-audio).


## Get a transcript of the audio file
To get a transcript, just ask for it in the prompt. For example:


In [ ]:
prompt = "Generate a transcript of the speech."

## Refer to timestamps in the audio file
A prompt can specify timestamps of the form `MM:SS` to refer to particular sections in an audio file. For example:

In [17]:
# Create a prompt containing timestamps.
prompt = "Provide a transcript of the speech from 02:30 to 03:29."

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    prompt,
    your_file,
  ]
)

print(response.text)

The President's State of the Union address to a joint session of the Congress from the rostrum of the House of Representatives Washington, D.C., January 30th, 1961.

Mr. Vice President, members of the Congress, it is a pleasure to return from whence I came.

You are among my oldest friends in Washington, and this house is my oldest home. It was here.

It was here more than 14 years ago that I first took the oath of Federal office. It was here for 14 years that I gained both knowledge and inspiration from members of both parties in both houses, from your wise and generous leaders, and from the pronouncements which I can vividly recall, sitting where you now sit, including the programs of two great presidents. The undimmed eloquence of Churchill, the soaring idealism of Nehru, the steadfast words of General DeGaulle. To speak from this same historic rostrum is a sobering experience. To be back among so many friends is a happy one. I am confident that that friendship will continue. Our Co

## Count audio tokens

You can count the number of tokens in your audio file using the [count_tokens](https://googleapis.github.io/python-genai/#count-tokens-and-compute-tokens) method.

Audio files have a fixed per second token rate (more details in the dedicated [count token quickstart](./Counting_Tokens.ipynb).

In [18]:
count_tokens_response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents=[your_file],
)

print("Audio file tokens:",count_tokens_response.total_tokens)

Audio file tokens: 83528


Youtube Example

In [19]:
from google.genai import types
from IPython.display import display, Markdown
youtube_url = "https://www.youtube.com/watch?v=RDOMKIw1aF4" # Repalce with the youtube url you want to analyze

prompt = """
    Analyze the following YouTube video content. Provide a concise summary covering:

    1.  **Main Thesis/Claim:** What is the central point the creator is making?
    2.  **Key Topics:** List the main subjects discussed, referencing specific examples or technologies mentioned (e.g., AI models, programming languages, projects).
    3.  **Call to Action:** Identify any explicit requests made to the viewer.
    4.  **Summary:** Provide a concise summary of the video content.

    Use the provided title, chapter timestamps/descriptions, and description text for your analysis.
"""
# Analyze the video
response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=types.Content(
        parts=[
            types.Part(text=prompt),
            types.Part(
                file_data=types.FileData(file_uri=youtube_url)
            )
        ]
    )
)
display(Markdown(response.text))

Okay, here's an analysis of the YouTube video content based on the provided title and visual/audio information:

1.  **Main Thesis/Claim:** Google's Gemini 2.5 Pro is the best AI model currently available specifically for coding tasks, particularly excelling in complex code generation and refactoring compared to other models the creator has tested.

2.  **Key Topics:**
    *   **Gemini 2.5 Pro Introduction:** Announcing the model and the creator's positive initial impression.
    *   **Code Generation Examples:**
        *   **Ultimate Tic-Tac-Toe:** Generating a complex game in Java using Swing, which worked correctly on the first attempt ("one-shot").
        *   **Kitten Cannon Clone:** Generating a launch-style game using p5.js (JavaScript). Required two debugging prompts ("three-shot") to fix errors (`oncontextmenu`, `sketch.sign` vs. `Math.sign`).
        *   **Landing Page (FishLog):** Attempting to build a landing page using Vite, React, and Tailwind CSS based on a mockup image. Performance was noted as poor.
        *   **X (Twitter) UI Clone:** Attempting to recreate a portion of the X website UI in a single HTML file. Performed slightly better than the landing page but still highlighted as not a strength.
    *   **Code Refactoring:** Demonstrating Gemini 2.5 Pro's ability to refactor the creator's "crappy" Rust code into a more idiomatic version using iterators, claiming it outperformed other models (like Claude 3.5 Sonnet) on this specific task.
    *   **Benchmark Analysis:** Reviewing benchmark charts (Humanity's Last Exam, GPQA diamond, AIME 2025, LiveCodeBench v5, Aider Polyglot, SWE-bench verified) comparing Gemini 2.5 Pro against models like OpenAI O3-mini, GPT-4.5, Claude 3.7 Sonnet, Grok 3 Beta, and DeepSeek R1. Highlights its strong reasoning, science, math (vs. most), and code editing capabilities, though not universally topping code generation benchmarks shown.
    *   **Knowledge Cutoff & Grounding:** Testing the model's knowledge of the current React.js version, initially showing outdated info (pre-Dec 2024) but correcting it when using the "Grounding with Google Search" feature. The benchmark slide indicated a March 2025 training data cutoff, creating a slight discrepancy.

3.  **Call to Action:** The creator explicitly asks viewers to share their own experiences and thoughts on Gemini 2.5 Pro, especially regarding coding, in the comments section. Standard engagement requests (like, subscribe) are also mentioned briefly at the very end.

4.  **Summary:** The video presents Google's Gemini 2.5 Pro as the leading AI for coding. The creator showcases its success in generating complex Java and p5.js games (though the latter needed debugging) and its superior ability to refactor Rust code compared to rivals. While acknowledging its limitations in front-end UI generation (demonstrated with React/Tailwind and HTML examples), the analysis of benchmarks and the impressive refactoring lead the creator to conclude it's currently the best overall *coding* AI, despite not topping every benchmark shown.

## Next Steps
### Useful API references:

More details about Gemini API's [vision capabilities](https://ai.google.dev/gemini-api/docs/vision) in the documentation.

If you want to know about the File API, check its [API reference](https://ai.google.dev/api/files) or the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) quickstart.

### Related examples

Check this example using the audio files to give you more ideas on what the gemini API can do with them:
* Share [Voice memos](https://github.com/google-gemini/cookbook/blob/main/examples/Voice_memos.ipynb) with Gemini API and brainstorm ideas

### Continue your discovery of the Gemini API

Have a look at the [Audio](../quickstarts/Audio.ipynb) quickstart to learn about another type of media file, then learn more about [prompting with media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length for audio files. .
